In [1]:
# ResNet18 CiFar10

import torch
import torchvision
from torch import nn
from torch.nn import functional as F
import time
from torchvision import transforms


In [2]:
class Timer:
    """记录多次运行时间"""
    def __init__(self):
        """Defined in :numref:`subsec_linear_model`"""
        self.times = []
        self.start()

    def start(self):
        """启动计时器"""
        self.tik = time.time()

    def stop(self):
        """停止计时器并将时间记录在列表中"""
        self.times.append(time.time() - self.tik)
        return self.times[-1]
    
    def sum(self):
        """返回时间总和"""
        return sum(self.times)


class Accumulator:
    """在n个变量上累加"""
    def __init__(self, n):
        """Defined in :numref:`sec_softmax_scratch`"""
        self.data = [0.0] * n

    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]

    def reset(self):
        self.data = [0.0] * len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]
    


def accuracy(y_hat, y):  #@save
    """计算预测正确的数量"""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.type(y.dtype) == y
    return float(cmp.type(y.dtype).sum())


def evaluate_accuracy_gpu(net, data_iter, device=None):
    """使用GPU计算模型在数据集上的精度

    Defined in :numref:`sec_lenet`"""
    if isinstance(net, nn.Module):
        net.eval()  # 设置为评估模式
        if not device:
            device = next(iter(net.parameters())).device
    # 正确预测的数量，总预测的数量
    metric = Accumulator(2)
    with torch.no_grad():
        for X, y in data_iter:
            if isinstance(X, list):
                # BERT微调所需的（之后将介绍）
                X = [x.to(device) for x in X]
            else:
                X = X.to(device)
            y = y.to(device)
            metric.add(accuracy(net(X), y), y.numel())
    return metric[0] / metric[1]


In [3]:
class Residual(nn.Module):  #@save
    def __init__(self, input_channels, num_channels,
                 use_1x1conv=False, strides=1):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, num_channels,
                               kernel_size=3, padding=1, stride=strides)
        self.conv2 = nn.Conv2d(num_channels, num_channels,
                               kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(input_channels, num_channels,
                                   kernel_size=1, stride=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.bn2 = nn.BatchNorm2d(num_channels)

    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        Y += X
        return F.relu(Y)

In [4]:
def resnet_block(input_channels, num_channels, num_residuals,
                 first_block=False):
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.append(Residual(input_channels, num_channels,
                                use_1x1conv=True, strides=2))
        else:
            blk.append(Residual(num_channels, num_channels))
    return blk

In [5]:
b1 = nn.Sequential(nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
                   nn.BatchNorm2d(64), nn.ReLU(),
                   # nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
                    )

In [6]:
b2 = nn.Sequential(*resnet_block(64, 64, 2, first_block=True))
b3 = nn.Sequential(*resnet_block(64, 128, 2))
b4 = nn.Sequential(*resnet_block(128, 256, 2))
b5 = nn.Sequential(*resnet_block(256, 512, 2))

In [7]:
net = nn.Sequential(b1, b2, b3, b4, b5,
                    nn.AdaptiveAvgPool2d((1,1)),
                    nn.Flatten(), nn.Linear(512, 10))

In [8]:
X = torch.rand(size=(1, 3, 32, 32))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t', X.shape)

Sequential output shape:	 torch.Size([1, 64, 32, 32])
Sequential output shape:	 torch.Size([1, 64, 32, 32])
Sequential output shape:	 torch.Size([1, 128, 16, 16])
Sequential output shape:	 torch.Size([1, 256, 8, 8])
Sequential output shape:	 torch.Size([1, 512, 4, 4])
AdaptiveAvgPool2d output shape:	 torch.Size([1, 512, 1, 1])
Flatten output shape:	 torch.Size([1, 512])
Linear output shape:	 torch.Size([1, 10])


In [9]:
trans = [transforms.RandomCrop(32, padding=4),
         transforms.RandomHorizontalFlip(p=0.5),
         transforms.RandomRotation(45),
         transforms.ToTensor()]
trans = transforms.Compose(trans)
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, transform=trans, download=True)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transforms.ToTensor(),
                                         download=True)

batch_size = 128
train_iter = torch.utils.data.DataLoader(train_data, batch_size, shuffle=True)
test_iter = torch.utils.data.DataLoader(test_data, batch_size, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [10]:
lr, num_epochs= 0.1, 200
def init_weights(m):
    if type(m) == nn.Linear or type(m) == nn.Conv2d:
        nn.init.xavier_uniform_(m.weight)
net.apply(init_weights)

device = torch.device('cuda:0')
print('training on', device)
net.to(device)
optimizer = torch.optim.SGD(net.parameters(), lr=lr)
loss = nn.CrossEntropyLoss()
timer, num_batches = Timer(), len(train_iter)
counter = 0
best_acc = 0.85
for epoch in range(num_epochs):
    counter += 1
    if counter / 5 == 1:
        counter = 0
        lr = lr * 0.5
    # 训练损失之和，训练准确率之和，样本数
    metric = Accumulator(3)
    net.train()
    for i, (X, y) in enumerate(train_iter):
        timer.start()
        optimizer.zero_grad()
        X, y = X.to(device), y.to(device)
        y_hat = net(X)
        l = loss(y_hat, y)
        l.backward()
        optimizer.step()
        with torch.no_grad():
            metric.add(l * X.shape[0], accuracy(y_hat, y), X.shape[0])
        timer.stop()
        train_l = metric[0] / metric[2]
        train_acc = metric[1] / metric[2]
        
    test_acc = evaluate_accuracy_gpu(net, test_iter)
    if test_acc > best_acc:
        best_acc = test_acc
        torch.save(net.state_dict(), 'resnet18.params')
        torch.save(best_acc, 'best_acc')
    print(f'train loss: {train_l}, train acc: {train_acc}, test acc: {test_acc}')
print(f'loss {train_l:.3f}, train acc {train_acc:.3f}, '
      f'test acc {test_acc:.3f}')
print(f'{metric[2] * num_epochs / timer.sum():.1f} examples/sec '
      f'on {str(device)}')
print(f' total time {timer.sum()}')

training on cuda:0
train loss: 1.8115907727050782, train acc: 0.36292, test acc: 0.3783
train loss: 1.3873056744384766, train acc: 0.49512, test acc: 0.4045
train loss: 1.2077794293212891, train acc: 0.56656, test acc: 0.5568
train loss: 1.0834905618286133, train acc: 0.61258, test acc: 0.6172
train loss: 0.9842662455749511, train acc: 0.64992, test acc: 0.6889
train loss: 0.894689280090332, train acc: 0.68432, test acc: 0.7066
train loss: 0.8265599975585938, train acc: 0.70888, test acc: 0.7145
train loss: 0.7731779887390137, train acc: 0.729, test acc: 0.6998
train loss: 0.7262418509674072, train acc: 0.7461, test acc: 0.7589
train loss: 0.6856836380004883, train acc: 0.76046, test acc: 0.7583
train loss: 0.6538150772857666, train acc: 0.77068, test acc: 0.7615
train loss: 0.6210008462524415, train acc: 0.78282, test acc: 0.7854
train loss: 0.5928880404663086, train acc: 0.79334, test acc: 0.7161
train loss: 0.5699223737335205, train acc: 0.80116, test acc: 0.7956
train loss: 0.54660

train loss: 0.06904429869174958, train acc: 0.97534, test acc: 0.8971
train loss: 0.06995185104846954, train acc: 0.97572, test acc: 0.8931
train loss: 0.0658977385544777, train acc: 0.9762, test acc: 0.8932
train loss: 0.06663578231334687, train acc: 0.97628, test acc: 0.8974
train loss: 0.061216841411590575, train acc: 0.97882, test acc: 0.894
train loss: 0.06576451285839081, train acc: 0.977, test acc: 0.8977
train loss: 0.0622458757853508, train acc: 0.97756, test acc: 0.8932
train loss: 0.06347101771116256, train acc: 0.9778, test acc: 0.8906
train loss: 0.06018660384893417, train acc: 0.9785, test acc: 0.894
train loss: 0.06280301968097687, train acc: 0.97762, test acc: 0.8987
train loss: 0.06585178529977799, train acc: 0.97682, test acc: 0.8906
train loss: 0.05975224113225937, train acc: 0.97854, test acc: 0.8996
train loss: 0.05897056128263473, train acc: 0.97828, test acc: 0.8989
train loss: 0.0554973889029026, train acc: 0.9803, test acc: 0.8938
train loss: 0.0591020968818664